## Demonstration of a Ring Buffer Baseline Recommender System
Here the demonstration of the ring buffer recommender is presented. It recommends items to the user based on which are present in the current *RingBuffer* (see implementation in *ring_buffer_baseline.py*). This way, it captures both recency of the items and the popularity through how many ring buffer entries each item has.

It simply recommends the first item which is not the one the user is currently browsing, found by looking back in the ring buffer.

The file also includes the evaluation of the recommender model using the metrics *Precision and Recall*.

In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
from parquet_data_reader import ParquetDataReader
from models.ring_buffer_baseline import RingBufferBaseline

import polars as pl
pl.Config.set_tbl_cols(-1)
import numpy as np
parquet_reader = ParquetDataReader()

### Data Extraction and Processing

In [2]:
import polars as pl
from utils.baseline_processing import process_behavior_data, random_split, time_based_split

train_behavior_df = parquet_reader.read_data("../../data/train/behaviors.parquet")
test_behaviours_df = parquet_reader.read_data('../../data/validation/behaviors.parquet')

# Processes the data
combined_df = process_behavior_data(train_behavior_df, test_behaviours_df)

# ----- Method 1: Random Split -----
train_random, test_random = random_split(combined_df, test_ratio=0.30)
print("Random Split:")
print("Train shape:", train_random.shape)
print("Test shape:", test_random.shape)

# ----- Method 2: Time-based Split -----
train_time, test_time = time_based_split(combined_df, test_ratio=0.30)
print("\nTime-based Split:")
print("Train shape:", train_time.shape)
print("Test shape:", test_time.shape)


Random Split:
Train shape: (99057, 4)
Test shape: (42666, 4)

Time-based Split:
Train shape: (99207, 4)
Test shape: (42516, 4)


### Method 1: Random Split of Train/Test for Recommendations

In [3]:
# Creates a recommender and fits it to the training data split using the random split method
recommender = RingBufferBaseline(behaviors=train_random)
recommender.fit()

user_id_test = 151570
recommendations = recommender.recommend(user_id=user_id_test, n=5)

print(f"Recommendations for user {user_id_test}:")
print(recommendations)

Recommendations for user 151570:
[9771042, 9770541, 9770882, 9769650, 9771042]


### Method 2: Time-based Split of Train/Test for Recommendations
This methods splits the data into the oldest interactions *(test_ratio percent)*
are used for testing, and the newest interactions are used for training. This happens after the total data (train and test) has been combined. 

In [4]:
# Creates a recommender and fits it to the training data split using the time-based split method
recommender2 = RingBufferBaseline(behaviors=train_time)
recommender2.fit()

user_id_test2 = 151570
recommendations2 = recommender.recommend(user_id=user_id_test2, n=5)

print(f"Recommendations for user {user_id_test2}:")
print(recommendations2)

Recommendations for user 151570:
[9771042, 9770541, 9770882, 9769650, 9771042]


### Comparison: Evaluation of the Ring Buffer Baseline Recommender
Comparing the two different data-splits for this  ring buffer baseline recommender using the metrics *Precision and Recall*.
*FPR* is also printed for reference.

In [5]:
from utils.evaluation import perform_model_evaluation

# Evaluates the recommender using the same random split data
metrics = perform_model_evaluation(recommender, test_data=test_random, k=5)
print("\nEvaluation metrics (precision and recall at k):")
print(metrics)


# Evaluates the recommender using time split data
metrics2 = perform_model_evaluation(recommender2, test_data=test_time, k=5)
print("\nEvaluation metrics (precision and recall at k):")
print(metrics2)



Evaluation metrics (precision and recall at k):
{'precision@k': np.float64(0.0015628439357252917), 'recall@k': np.float64(0.0019321484546187342), 'fpr@k': np.float64(0.002190689623935496)}

Evaluation metrics (precision and recall at k):
{'precision@k': np.float64(0.01882155822395979), 'recall@k': np.float64(0.03013248050153789), 'fpr@k': np.float64(0.004282433544948659)}


In [6]:
from utils.evaluation import append_model_metrics
append_model_metrics(metrics, "ring_buffer_baseline_random_split")
append_model_metrics(metrics2, "ring_buffer_baseline_time_split")

### Carbon Footprint
This section creates an emissions.csv file in the "output"-folder
It utilizes the code_carbon (`codecarbon EmissionsTracker`) to record the carbon footprint of the `fit` and the `recommend` methods of the model.

In [7]:
from utils.evaluation import track_model_energy

print("\nCarbon footprint of the recommender:")
footprint = track_model_energy(recommender, "ring_buffer", user_id=user_id_test2, n=5)
footprint

[codecarbon INFO @ 11:03:24] [setup] RAM Tracking...
[codecarbon INFO @ 11:03:24] [setup] CPU Tracking...
[codecarbon WARNING @ 11:03:24] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU




Carbon footprint of the recommender:


[codecarbon WARNING @ 11:03:26] We saw that you have a 13th Gen Intel(R) Core(TM) i7-13700H but we don't know it. Please contact us.
[codecarbon INFO @ 11:03:26] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 11:03:26] [setup] GPU Tracking...
[codecarbon INFO @ 11:03:26] No GPU found.
[codecarbon INFO @ 11:03:26] >>> Tracker's metadata:
[codecarbon INFO @ 11:03:26]   Platform system: Windows-10-10.0.26100-SP0
[codecarbon INFO @ 11:03:26]   Python version: 3.11.9
[codecarbon INFO @ 11:03:26]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 11:03:26]   Available RAM : 15.731 GB
[codecarbon INFO @ 11:03:26]   CPU count: 20
[codecarbon INFO @ 11:03:26]   CPU model: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 11:03:26]   GPU count: None
[codecarbon INFO @ 11:03:26]   GPU model: None
[codecarbon INFO @ 11:03:29] Saving emissions data to file c:\Users\magnu\NewDesk\An.sys\TDT4215\recommender_system\demostrations\output\ring_buffer_fit

{'fit': (None, 3.141829992693206e-08),
 'recommend': ([9771042, 9770541, 9770882, 9769650, 9771042],
  4.574156998942372e-09)}